In [1]:
import time
import pandas as pd
import spacy
import en_core_web_sm
from spacy import displacy
from collections import Counter

nlp = en_core_web_sm.load()

data_raw = pd.read_csv('pictures_sample.csv')

data = data_raw[['PreviewMediumUri','Description']]

c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3,21,22,33,35,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
k = 0
data.head()

,PreviewMediumUri,Description
0,http://c9.mediaforum.afp.com/CacheServer/Cache...,US-British actress Sienna Miller poses after u...
1,http://c12.mediaforum.afp.com/CacheServer/Cach...,US-British actress Sienna Miller poses after u...
2,http://c5.mediaforum.afp.com/CacheServer/Cache...,People gather at a shopping mall in the Shatin...
3,http://c2.mediaforum.afp.com/CacheServer/Cache...,AFP presents a photo essay of 19 images by pho...
4,http://c11.mediaforum.afp.com/CacheServer/Cach...,A Palestinian worker harvests dates in the Jor...


In [33]:
data['PreviewMediumUri'][0]

'http://c9.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=CF78B67020F78E238CF5F11D170DF6188316DC7B42ACFF02EB50F3A6859CB438'

In [3]:
column = 'Description'
copy = data.copy()
copy_column = copy[column]
entities = []
total = 0
while(k < 1):
    start = time.process_time()
    for text in copy_column[200*k:200*(k+1)]:
        text_ent =  nlp(text)
        entities.append([(X.text, X.label_) for X in text_ent.ents])
    k += 1
    end = time.process_time() - start
    total += end
    print(k, 'ère'*(k == 1) + 'ème'*(k != 1) + " itération. Temps écoulé pour l'étape: ", end, ' secondes. Temps écoulé total: ', total, ' secondes.', sep = '')

1ère itération. Temps écoulé pour l'étape: 3.703125 secondes. Temps écoulé total: 3.703125 secondes.


In [4]:
print(entities)

[[('US', 'GPE'),
  ('Sienna Miller', 'PERSON'),
  ('45th', 'ORDINAL'),
  ('Deauville US Film Festival', 'EVENT'),
  ('Deauville', 'GPE'),
  ('France', 'GPE'),
  ('September 11, 2019', 'DATE')],
 [('US', 'GPE'),
  ('Sienna Miller', 'PERSON'),
  ('45th', 'ORDINAL'),
  ('Deauville US Film Festival', 'EVENT'),
  ('Deauville', 'GPE'),
  ('France', 'GPE'),
  ('September 11, 2019', 'DATE')],
 [('Shatin', 'LOC'),
  ('Hong Kong', 'GPE'),
  ('September 11, 2019', 'DATE'),
  ('Hong Kong', 'GPE'),
  ('Nicolas ASFOURI / AFP', 'PERSON')],
 [('AFP', 'ORG'),
  ('19', 'CARDINAL'),
  ('Christof Stache', 'PERSON'),
  ('September 11, 2019', 'DATE'),
  ('Bad Hindelang', 'FAC')],
 [('Palestinian', 'NORP'),
  ('Jordan Valley', 'LOC'),
  ('Jiftlik', 'PERSON'),
  ('Israeli', 'NORP'),
  ('West Bank', 'GPE'),
  ('September 11, 2019', 'DATE'),
  ('Israeli', 'NORP'),
  ('Benjamin Netanyahu', 'PERSON'),
  ("the West Bank's", 'GPE'),
  ('Jordan Valley', 'LOC'),
  ('today', 'DATE'),
  ('September 17', 'DATE'),
  ('Ne

In [53]:
def similaire(test, texte):
    intersection = set(test.lower().split(" ")).intersection(set(texte.lower().split(" ")))
    union = set(test.lower().split(" ")).union(set(texte.lower().split(" ")))
    return(len(intersection)/len(union))

def index_prob(text):
    text_nlp = nlp(text)
    text_entities = [(X.text, X.label_) for X in text_nlp.ents]
    score_s = []
    score = []
    for l in entities:
        score_j = 0
        score_l = 0
        for k in l:
            for ent in text_entities:
                if(k[1] == ent[1]):
                    score_j += similaire(k[0], ent[0])
                    score_a = [(mot.lower() in ent[0].lower()) for mot in k[0].split(" ")]
                    score_b = [(mot.lower() in k[0].lower()) for mot in ent[0].split(" ")] # pas nécessaire
                    score_l += (np.sum(score_a) >= 1)*(np.sum(score_b) >= 1)
        score.append(score_l)
        score_s.append(score_j)
    score = [(score[i], i) for i in range(len(score))]
    score_s = [(score_s[i], i) for i in range(len(score_s))]
    score.sort(reverse = True, key = lambda tup: tup[0])
    score_s.sort(reverse = True, key = lambda tup: tup[0])
    return(score,score_s)

def retour_image(liste):
    for i in range(5):
        print(data['PreviewMediumUri'][liste[i][1]])

In [55]:
test = index_prob("On the 21th of September, Donald Trump said something that might have been important, or not.")
retour_image(test[0])
print('\n')
retour_image(test[1])

http://c12.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56275115DE80A0837E30F2DDF42420024EBB798FB7B269A8D5E6
http://c12.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56275115DE80A0837E30F2DDF42420024EBB798FB7B269A8D5E6
http://c5.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56277BF007D9CF94D677F9B3983096A36A3630DF636012D98EED
http://c11.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=A82E11DF70F5E42D935B14E0A1037337BB580A0F6BFCACD1A8F140587C6A2394
http://c12.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56270CC64A8A5A2D8E4217D1C250FECACAF8D59F7E99F9F82D83


http://c12.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56275115DE80A0837E30F2DDF42420024EBB798FB7B269A8D5E6
http://c12.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56275115DE80A0837E30F2DDF42420024EBB798FB7B269A8D5E6
http://c5.mediaforum.afp.com/CacheServer/Cache.svc/g?hid=05E22585D39F56277BF007D9CF94D677F9B3983096A36A3630DF636012D98EED
http://c8.mediaf

In [56]:
nlp("On the 21th of September, Donald Trump said something that might have been important, or not.").ents
# Mettre des poids différents sur les entités nommées ?

(the 21th of September, Donald Trump)

In [60]:
data_raw.dtypes

Unnamed: 0                       int64
AfpForumIntroductionUtcDate     object
AfpForumUpdateUtcDate           object
AudioDescription                object
BinaryVersion                  float64
CaptionWriter                   object
CategoryCodes                   object
City                            object
ContentCreationUtcDate          object
ContentLanguage                 object
Contributors                    object
Country                         object
CountryCode                     object
Credit                          object
CreditLine                      object
Description                     object
DescriptorVersion                int64
DocumentFormat                  object
Duration                         int64
EmbargoUtcDate                 float64
EventsId                        object
GenreAttribute                  object
GenreTypes                      object
HasTitling                        bool
HighResHeight                  float64
HighResWidth             

In [63]:
data_raw['Keywords']

0         ['SMILING', 'INAUGURATION', 'SUNGLASSES', 'FIL...
1         ['FLAG', 'SMILING', 'INAUGURATION', 'SUNGLASSE...
2                                    ['POLITICS', 'UNREST']
3                              ['AGRICULTURE', 'TRADITION']
4                                                        []
5                                             ['DIPLOMACY']
6         ['CAR', 'ILLUSTRATION', 'AUTO INDUSTRY', 'ELEC...
7                                             ['TRANSPORT']
8                        ['RELIGION', 'ISLAM', 'TRADITION']
9                                            ['LITERATURE']
10        ['SMILING', 'INAUGURATION', 'SUNGLASSES', 'FIL...
11        ['FLAG', 'SMILING', 'INAUGURATION', 'SUNGLASSE...
12                                   ['POLITICS', 'UNREST']
13                             ['AGRICULTURE', 'TRADITION']
14                                                       []
15                                            ['DIPLOMACY']
16        ['CAR', 'ILLUSTRATION', 'AUTO 

In [65]:
data_raw['City']

0                 Deauville
1                 Deauville
2                 Hong Kong
3             Bad Hindelang
4             Jordan Valley
5                    Ankara
6         Frankfurt am Main
7         Frankfurt am Main
8                  Nabatieh
9                    London
10                Deauville
11                Deauville
12                Hong Kong
13            Bad Hindelang
14            Jordan Valley
15                   Ankara
16        Frankfurt am Main
17        Frankfurt am Main
18                 Nabatieh
19                   London
20                   London
21                 Nabatieh
22                   London
23                Singapore
24                 Hamilton
25                 New York
26                    Busan
27                 New York
28                 New York
29            Beverly Hills
                ...        
221734               La Paz
221735               La Paz
221736               La Paz
221737               La Paz
221738              

In [96]:
def score_loc(texte):
    texte_nlp = nlp(texte)
    loc_ent = [i.text for i in texte_nlp.ents if i.label_ == 'GPE']
    score = [0 for _i in range(len(entities))]
    for i in loc_ent:
        l1 = [1*(j in i) if type(j) != float else 0 for j in data_raw['Country']]
        l2 = [1*(j in i) if type(j) != float else 0 for j in data_raw['City']]
        score = [score[i]+l1[i]+l2[i] for i in range(len(entities))]
    score = [(score[i], i) for i in range(len(score))]
    score.sort(reverse = True, key = lambda tup: tup[0])
    return(score)

In [97]:
score_loc("This is a beautiful day in France right?")

[(1, 0),
 (1, 1),
 (1, 10),
 (1, 11),
 (1, 42),
 (1, 44),
 (1, 52),
 (1, 68),
 (1, 70),
 (1, 71),
 (1, 72),
 (1, 92),
 (1, 100),
 (1, 101),
 (1, 102),
 (1, 118),
 (1, 119),
 (1, 120),
 (1, 122),
 (1, 129),
 (1, 131),
 (1, 132),
 (1, 136),
 (1, 137),
 (1, 138),
 (1, 141),
 (1, 143),
 (1, 150),
 (1, 153),
 (1, 156),
 (1, 157),
 (1, 158),
 (1, 159),
 (1, 160),
 (1, 161),
 (1, 162),
 (1, 163),
 (1, 164),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (0, 43),
 (0, 45),
 (0, 46),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 51),
 (0, 53),
 (0, 54),
 (0, 55),
 (0, 56),
 (0, 57),
 (0, 58),
 (0, 59),
 (0, 60),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 64),
 (0, 65),
 (0, 66),
 (0,